In [1]:
import torch
from PIL import Image
from transformers import AutoProcessor, CLIPModel
import torch.nn as nn
import requests
import glob
import json
import numpy as np
import torch.nn.functional as F

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
%cd /content/drive/My Drive/xgen_imgs

/content/drive/My Drive/xgen_imgs


use pretrained CLIP to get embeddings.

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [6]:
img_paths = sorted(glob.glob("*.jpg"))
print(len(img_paths))

2652


load each image, process it as input, run it through pretrained model to get embeddings

In [7]:
res = []
cnt = 0
for p in img_paths:
  img = Image.open(p)
  with torch.no_grad():
      inputs = processor(images=img, return_tensors="pt").to(device)
      image_features = model.get_image_features(**inputs)
  res.append(image_features)
  if cnt % 100 ==0:
    print("processed ", cnt)
  cnt += 1

processed  0
processed  100
processed  200
processed  300
processed  400
processed  500
processed  600
processed  700
processed  800
processed  900
processed  1000
processed  1100
processed  1200
processed  1300
processed  1400
processed  1500
processed  1600
processed  1700
processed  1800
processed  1900
processed  2000
processed  2100
processed  2200
processed  2300
processed  2400
processed  2500
processed  2600


concat embeddings to get N x 512 sized matrix

In [8]:
catalog = torch.cat(res, axis=0)

normalize embeddings per row. do this to make it easier to compute cosine similarity later

In [9]:
catalog = F.normalize(catalog, dim=1)

In [10]:
torch.save(catalog, 'catalog.pt')
with open('catalog.npy', 'wb') as f:
    np.save(f, catalog.numpy())

In [11]:
cat = np.load("catalog.npy")
cat

array([[-0.01277409, -0.02755296,  0.06707464, ...,  0.01357371,
         0.04564205, -0.00260423],
       [-0.00393393,  0.00528876,  0.0331343 , ...,  0.05045484,
         0.0323632 ,  0.04916117],
       [-0.02948941,  0.02274247,  0.04598304, ...,  0.03811409,
         0.02792593,  0.04891756],
       ...,
       [ 0.0069792 ,  0.00821297,  0.01164407, ...,  0.05097448,
        -0.00574096,  0.04718106],
       [-0.01457813, -0.06095503,  0.03823836, ...,  0.0230542 ,
         0.03104806,  0.03030955],
       [-0.01196348, -0.04884604,  0.0283636 , ...,  0.04764979,
        -0.00982623,  0.06089049]], dtype=float32)